In [1]:
%pip install mne numpy scipy scikit-learn tensorflow keras colorlog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import utils
import mne
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score


2024-07-24 13:48:54.288755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 13:48:54.289062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 13:48:54.309540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 13:48:54.373896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 13:48:56.628748: W tensorflow/comp

# Neural networks
This notebook explores the usability of neural network architectures on the cleaned rsEEG data. This doesn't make use of the extracted features, which means that there is a larger chance of finding patterns in the raw data and might result in better generalizability to other patients.

In [3]:
labels_df = utils.get_metadata_df("dataset-cleaned", "Randomisatielijst.csv")
labels_df

,filename,procedure,patient_id,eeg_type,pre_post
0,TMS-EEG-H_06_S3_spTEP_post-epo.fif,sham,06,spTEP,post
1,TMS-EEG-H_14_S2_rsEEG_post-epo.fif,itbs,14,rsEEG,post
2,TMS-EEG-H_15_S1_spTEP_pre-epo.fif,ctbs,15,spTEP,pre
3,TMS-EEG-H_08_S2_rsEEG_pre-epo.fif,itbs,08,rsEEG,pre
4,TMS-EEG-H_16_S2_spTEP_pre-epo.fif,sham,16,spTEP,pre
...,...,...,...,...,...
173,TMS-EEG-H_15_S3_spTEP_post-epo.fif,itbs,15,spTEP,post
174,TMS-EEG-H_07_S2_rsEEG_post-epo.fif,ctbs,07,rsEEG,post
175,TMS-EEG-H_13_S3_spTEP_post-epo.fif,sham,13,spTEP,post
176,TMS-EEG-H_08_s1_spTEP_post-epo.fif,sham,08,spTEP,post


In [4]:
filtered_df = labels_df[
    (labels_df['eeg_type'] == 'rsEEG')
    & (labels_df['procedure'] == 'itbs')
    & (labels_df['patient_id'].isin(['08', '09', '10']))]
epoch_files = filtered_df['filename'].tolist()
epochs_list = [mne.read_epochs(os.path.join("dataset-cleaned", file)) for file in epoch_files]
all_epochs = mne.concatenate_epochs(epochs_list)

# Training data & labels
X = all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
y = np.array([1 if timing == 'pre' else 0 for timing in filtered_df['pre_post']])

# Fix length of data with labels
if len(X) != len(y):
    y = np.repeat(y, len(X) // len(y) + 1)[:len(X)]
if len(X) != len(y):
    raise ValueError(f"Data cardinality is ambiguous: x sizes: {len(X)}, y sizes: {len(y)}")

Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_08_S2_rsEEG_pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_08_S2_rsEEG_post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
317 matching events found
No baseline correction applied
0 projection items activated
Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_10_S2_rsEEG_post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
185 matching events found
No baseline correction applied
0 projection items activated
Readi

/tmp/ipykernel_342/1032790085.py:6: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
1424 matching events found
Applying baseline correction (mode: mean)


/tmp/ipykernel_342/1032790085.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


In [7]:
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),
    
    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),
    
    Conv1D(256, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
45/45 [==============================] - 7s 20ms/step - loss: 0.8098 - accuracy: 0.5400
Epoch 2/10
45/45 [==============================] - 1s 16ms/step - loss: 0.7168 - accuracy: 0.5765
Epoch 3/10
45/45 [==============================] - 1s 14ms/step - loss: 0.6854 - accuracy: 0.5709
Epoch 4/10
45/45 [==============================] - 1s 15ms/step - loss: 0.6661 - accuracy: 0.5899
Epoch 5/10
45/45 [==============================] - 1s 14ms/step - loss: 0.6649 - accuracy: 0.5934
Epoch 6/10
45/45 [==============================] - 1s 14ms/step - loss: 0.6460 - accuracy: 0.6011
Epoch 7/10
45/45 [==============================] - 1s 16ms/step - loss: 0.6420 - accuracy: 0.6285
Epoch 8/10
45/45 [==============================] - 1s 14ms/step - loss: 0.6266 - accuracy: 0.6348
Epoch 9/10
45/45 [==============================] - 1s 14ms/step - loss: 0.6138 - accuracy: 0.6552
Epoch 10/10
45/45 [==============================] - 1s 14ms/step - loss: 0.5797 - accuracy: 0.7008


In [5]:
X_CNN = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Shape: (n_epochs, n_channels, n_times, 1)

model_CNN = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3])),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_CNN.fit(X_CNN, y, epochs=10, batch_size=32)

2024-07-21 17:31:19.067788: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10532 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/10


2024-07-21 17:31:23.800215: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-07-21 17:31:24.131163: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-07-21 17:31:28.749684: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fecc42fd3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-21 17:31:28.749732: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-07-21 17:31:28.766518: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1721583088.950225     304 device_compiler.h:186] Compiled cluster using

45/45 [==============================] - 32s 470ms/step - loss: 1.5020 - accuracy: 0.5133
Epoch 2/10
45/45 [==============================] - 19s 429ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 3/10
45/45 [==============================] - 19s 429ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 4/10
45/45 [==============================] - 19s 430ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 5/10
45/45 [==============================] - 19s 428ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 6/10
45/45 [==============================] - 19s 429ms/step - loss: 0.6932 - accuracy: 0.5014
Epoch 7/10
10/45 [=====>........................] - ETA: 15s - loss: 0.6933 - accuracy: 0.4750

KeyboardInterrupt: 

In [56]:
new_patient_id = '11'  # Change this to the desired patient_id
new_filtered_df = labels_df[(labels_df['procedure'] == 'itbs') & (labels_df['eeg_type'] == 'rsEEG') & (labels_df['patient_id'] == new_patient_id)]
new_epoch_files = new_filtered_df['filename'].tolist()
new_epochs_list = [mne.read_epochs(os.path.join("dataset-cleaned", file)) for file in new_epoch_files]
new_all_epochs = mne.concatenate_epochs(new_epochs_list)

X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
y_test = np.array([1 if timing == 'pre' else 0 for timing in new_filtered_df['pre_post']])

# Ensure y has the same number of samples as X
if len(X_test) != len(y_test):
    y_test = np.repeat(y_test, len(X_test) // len(y_test) + 1)[:len(X_test)]

# Check if the lengths match
if len(X_test) != len(y_test):
    raise ValueError(f"Data cardinality is ambiguous: x sizes: {len(X_test)}, y sizes: {len(y_test)}")

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).flatten()

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy for patient {new_patient_id}: {accuracy}')

Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_11_S1_rsEEG_pre-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
255 matching events found
No baseline correction applied
0 projection items activated
Reading /project_ghent/Tomas_research/tms-research/dataset-cleaned/TMS-EEG-H_11_S1_rsEEG_post-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_1305/1814996412.py:5: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  new_all_epochs = mne.concatenate_epochs(new_epochs_list)


Not setting metadata
543 matching events found
Applying baseline correction (mode: mean)


/tmp/ipykernel_1305/1814996412.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_test = new_all_epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


17/17 [==============================] - 0s 8ms/step
Accuracy for patient 11: 0.5009208103130756
